# A* Pathfinding v1: Spatial Index Environment

This notebook demonstrates the use of the `SpatialIndexEnvironment` for pathfinding on point cloud data.

## Setup

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add project root to path
project_root = Path("../").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.terrain_navigate import AStar, SpatialIndexEnvironment, EuclideanCost

## 1. Load Data
We load the processed grid and convert it back to points for the Spatial Index demo.

In [ ]:
data_path = Path("data/processed_map.npy")
if not data_path.exists():
    raise FileNotFoundError("processed_map.npy not found. Please run preprocess_data.py first.")

Z = np.load(data_path)
print(f"Loaded map with shape: {Z.shape}")

# Convert grid to points (x, y, z)
y_coords, x_coords = np.indices(Z.shape)
points = np.column_stack((x_coords.ravel(), y_coords.ravel(), Z.ravel()))

# Subsample for performance in this demo if too large
if len(points) > 10000:
    print("Subsampling points for demo performance...")
    indices = np.random.choice(len(points), 10000, replace=False)
    points = points[indices]

print(f"Working with {len(points)} points")

plt.figure(figsize=(8, 6))
plt.scatter(points[:, 0], points[:, 1], c=points[:, 2], cmap='terrain', s=5)
plt.colorbar(label='Elevation (m)')
plt.title("Point Cloud from Terrain")
plt.show()

## 2. Initialize Spatial Index Environment
The `SpatialIndexEnvironment` builds a KDTree (or similar structure) to efficiently query nearest neighbors.

In [ ]:
env = SpatialIndexEnvironment(points)
cost_fn = EuclideanCost()
astar = AStar()

## 3. Plan Path

In [ ]:
# Define start and goal (relative to grid size)
max_x, max_y = Z.shape[1], Z.shape[0]
start_raw = np.array([max_x * 0.1, max_y * 0.1, 0.0])
goal_raw = np.array([max_x * 0.9, max_y * 0.9, 0.0])

# Snap to nearest existing points
start_node = env.get_nearest_node(start_raw)
goal_node = env.get_nearest_node(goal_raw)

print(f"Start Node: {start_node}")
print(f"Goal Node: {goal_node}")

# Find path with a connection radius
path, cost = astar.find_path(start_node, goal_node, env, cost_fn, node_radius=5.0)

if path is None:
    print("No path found!")
else:
    print(f"Path found! Length: {len(path)} nodes, Cost: {cost:.2f}")

## 4. Visualize Path

In [ ]:
if path is not None:
    path_arr = np.array(path)
    
    plt.figure(figsize=(10, 8))
    plt.scatter(points[:, 0], points[:, 1], c=points[:, 2], cmap='terrain', s=5, alpha=0.5)
    plt.colorbar(label='Elevation (m)')
    
    plt.plot(path_arr[:, 0], path_arr[:, 1], 'r-', linewidth=2, label='Path')
    plt.plot(start_node[0], start_node[1], 'go', markersize=10, label='Start')
    plt.plot(goal_node[0], goal_node[1], 'rx', markersize=10, label='Goal')
    
    plt.legend()
    plt.title("A* Path on Point Cloud")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.show()